In [7]:
import cv2
import numpy as np
import requests
# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
# بيانات API لخدمة Pushover
pushover_user_key = "ug6mt23dkm7trtwpnxjgzpq2p25irg"
pushover_api_token = "anegpdd71sa9wt5395bm9ssy4zmraj"

# دالة لإرسال إشعار عبر Pushover
def send_pushover_notification(message):
    payload = {
        "token": pushover_api_token,
        "user": pushover_user_key,
        "message": message,
    }
    requests.post("https://api.pushover.net/1/messages.json", data=payload)  # إرسال الطلب

# Initialize the webcam
cap = cv2.VideoCapture(0)

def get_posture(box):
    x, y, w, h = box
    aspect_ratio = h / w

    if aspect_ratio > 1.5:  # Heuristic for standing
        return "sitting"
    elif aspect_ratio < 0.5:  # Heuristic for fallen
        return "fallen"
    else:
        return "fallen"

while True:
    _, frame = cap.read()
    height, width, channels = frame.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:  # Class ID 0 is for 'person'
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)  # Green for the bounding box
            posture = get_posture(boxes[i])
            if posture == "fallen":
                color = (0, 0, 255)  # Red for fallen
                send_pushover_notification("A person has fallen!")  # إرسال إشعار بالسقوط
            elif posture == "sitting":
                color = (255, 0, 0)  # Blue for sitting
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{posture} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    if key == 27:  # Press 'ESC' to exit
        break

cap.release()
cv2.destroyAllWindows()

In [2]:
import re

pattern = re.compile(r"""
(?P<name>.+?)\s\[
(?P<image_link>https://cloudinary\.images-iherb\.com/image/upload/.+?)\]\sأضف\sإلى\sالسلة\s
(?P<name_repeat>.+?)\s\[
(?P<main_link>https://sa\.iherb\.com/r/.+?)\]
(?P<reviews>[\d,]+)?\s
\[
(?P<additional_link>https://sa\.iherb\.com/r/.+?)\]
(?:\sالمزيد\sمن\sالخيارات\sمتاحة\s
\[(?P<options_link>https://sa\.iherb\.com/pr/.+?)\])?\s
(?P<price>[\d.]+SAR)(?:\s(?P<original_price>[\d.]+SAR))?(?:\s(?P<discount>[\d%]+%\sخصم))?
""", re.VERBOSE)

# Example HTML content
html_content = """
ناو فودز‏, زيت السمك الغني بأوميجا-3 الفائق، 180 كبسولة هلامية [https://cloudinary.images-iherb.com/image/upload/f_auto,q_auto:eco/images/mli/mli00952/u/218.jpg] أضف إلى السلة كاليفورنيا غولد نيوتريشن‏, زيت السمك الممتاز بأوميجا 3، 100 كبسولة هلامية من جيلاتين السمك [https://sa.iherb.com/r/california-gold-nutrition-omega-3-premium-fish-oil-100-fish-gelatin-softgels/62118]372,211 [https://sa.iherb.com/r/california-gold-nutrition-omega-3-premium-fish-oil-100-fish-gelatin-softgels/62118] المزيد من الخيارات متاحة [https://sa.iherb.com/pr/california-gold-nutrition-omega-3-premium-fish-oil-100-fish-gelatin-softgels/62118] 24.54SAR 49.08SAR 50% خصم مميز! 
"""

# Extracting product information using the pattern
products = pattern.finditer(html_content)

for product in products:
    name = product.group('name')
    image_link = product.group('image_link')
    main_link = product.group('main_link')
    reviews = product.group('reviews')
    additional_link = product.group('additional_link')
    options_link = product.group('options_link')
    price = product.group('price')
    original_price = product.group('original_price')
    discount = product.group('discount')
    
    print(f"اسم المنتج: {name}")
    print(f"رابط الصورة: {image_link}")
    print(f"الرابط الرئيسي للمنتج: {main_link}")
    print(f"عدد التقييمات: {reviews}")
    print(f"رابط إضافي للمنتج: {additional_link}")
    print(f"رابط خيارات إضافية: {options_link}")
    print(f"السعر: {price}")
    print(f"السعر الأصلي: {original_price}")
    print(f"الخصم: {discount}")
    print("-" * 50)


اسم المنتج: ناو فودز‏, زيت السمك الغني بأوميجا-3 الفائق، 180 كبسولة هلامية
رابط الصورة: https://cloudinary.images-iherb.com/image/upload/f_auto,q_auto:eco/images/mli/mli00952/u/218.jpg
الرابط الرئيسي للمنتج: https://sa.iherb.com/r/california-gold-nutrition-omega-3-premium-fish-oil-100-fish-gelatin-softgels/62118
عدد التقييمات: 372,211
رابط إضافي للمنتج: https://sa.iherb.com/r/california-gold-nutrition-omega-3-premium-fish-oil-100-fish-gelatin-softgels/62118
رابط خيارات إضافية: https://sa.iherb.com/pr/california-gold-nutrition-omega-3-premium-fish-oil-100-fish-gelatin-softgels/62118
السعر: 24.54SAR
السعر الأصلي: 49.08SAR
الخصم: 50% خصم
--------------------------------------------------
